In [ ]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [ ]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
x, y = data.sample(d, 5)

In [ ]:
# dirrs = ['sentiment/train/', 'sentiment/test/']
# sent = []
# for dirr in dirrs:
#     print dirr
#     l = listdir(dirr+'pos')
#     print 'pos'
#     for r in l:
#         t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))
#     l = listdir(dirr+'neg')
#     print 'neg'
#     for r in l:
#         t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
#         sent.append(nltk.word_tokenize(t))

In [ ]:
# m = Word2Vec.load('embedding.model')
# data = []
# for s in sent:
#     a = [m[x] for x in s]
#     data.append(a)
# data = np.array(data)

In [ ]:
# def sam(d):
#     r = randint(0, data.shape[0] - 1)
#     t = d[r]
#     x = t[:-1]
#     y = t[1:]
#     return np.array(x), np.array(y)

# s = sample(data)
# s[0].shape, s[1].shape

In [ ]:
# hyper-parameters
epochs = 16000
hidden_layer = 128
inp_out_size = 324
learning_rate = 0.1
num_steps = 10

In [ ]:
# Wxh = tf.Variable([[.1, .2], [.3, .4], [.5, .6], [.7, .8]], dtype=tf.float32)
# Whh = tf.Variable([[.1, .2], [.3, .4]], dtype=tf.float32)
# Why = tf.Variable([[.1, .2, .3, .4], [.4, .5, .6, .7]], dtype=tf.float32)
Wxh = tf.Variable(tf.random_normal([inp_out_size, hidden_layer], mean=0, stddev=0.001))
Whh = tf.Variable(tf.random_normal([hidden_layer, hidden_layer], mean=0, stddev=0.001))
Why = tf.Variable(tf.random_normal([hidden_layer, inp_out_size], mean=0, stddev=0.001))
bh = tf.Variable(tf.zeros([hidden_layer]), dtype=tf.float32)
by = tf.Variable(tf.zeros([vocab_size]), dtype=tf.float32)

In [ ]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    h = tf.tanh((tf.matmul(p, Whh))  + tf.matmul(i, Wxh) + bh)
    h = tf.reshape(h, [hidden_layer])
    return h

In [ ]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.AdagradOptimizer(0.1)
optimize_op = optimizer.minimize(loss)

In [ ]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print gen
    print ' '.join(gen)

In [ ]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [ ]:
print epochs
for i in range(epochs):
    x, y = data.sample(d, num_steps)
    h = np.zeros(hidden_layer)
    l, o, _ = sess.run([loss, states, optimize_op], {a: x, b: y, initial: h})
    if i % 3000 == 0:
        print 'epoch {0}, loss = {1}'.format(i, l)
        print o[0]
        
#         generate(sess,40)

In [ ]:
generate(sess, 50)